In [2]:
import json
import os
from dotenv import load_dotenv
load_dotenv()
import threading
from collections import deque
from typing import Optional
from datetime import datetime, timezone, timedelta
from pathlib import Path

In [12]:
# (Reprendre les mêmes configs Azure que ci-dessus)
AZURE_SEARCH_ENDPOINT = "https://vectordbfundidcardfree.search.windows.net"
AZURE_SEARCH_KEY = os.getenv("AZURE_SEARCH_KEY")
INDEX_NAME = "rag-index"

AZURE_OPENAI_ENDPOINT = "https://alexa-me2hz4ri-swedencentral.cognitiveservices.azure.com/"
AZURE_OPENAI_KEY = os.getenv("OPENAI_KEY")
EMBEDDING_MODEL = "text-embedding-3-small"

# Pour le LLM (Génération) - On garde Ollama local ou on passe sur Azure GPT
# Ici je garde Ollama pour minimiser les changements, mais on peut switcher.
OLLAMA_BASE_URL = "http://localhost:11434"
MODEL_NAME = "mistral" 

In [ ]:
def _call_ollama(prompt):
        # (Même code que votre ancien RAGAppKafka)
        try:
            resp = requests.post(
                f"{OLLAMA_BASE_URL}/api/generate",
                json={"model": MODEL_NAME, "prompt": prompt, "stream": False}
            )
            return resp.json().get("response", "Erreur modèle")
        except Exception as e:
            return f"Erreur Ollama: {e}"

In [ ]:
from dotenv import load_dotenv
import os
from openai import AzureOpenAI
load_dotenv()

MODELS = {
    "gpt-4.1": {
        "deployment": "gpt-4.1",
        "endpoint": "https://al-mculmlvr-eastus2.cognitiveservices.azure.com/",
        "api_version": "2024-12-01-preview"
    },
    "model-router": {
        "deployment": "model-router",
        "endpoint": "https://alexa-me2hz4ri-swedencentral.cognitiveservices.azure.com/",
        "api_version": "2024-12-01-preview"
    },
}

SUBSCRIPTION_KEY = os.getenv("OPENAI_KEY")

In [7]:
def call_gpt_with_context(context: str, query: str, attachment: str, model_name="model-router") -> str:
    """
    Appelle Azure OpenAI en envoyant un prompt qui inclut :
    - un contexte
    - une requête
    - une pièce attachée (par ex. transcript YouTube)
    """
    config = MODELS.get(model_name)
    if not config:
        raise ValueError(f"Model '{model_name}' not configured")

    client = AzureOpenAI(
        api_version=config["api_version"],
        azure_endpoint=config["endpoint"],
        api_key=SUBSCRIPTION_KEY,
    )

    prompt = f"""
    You are an assistant.

    Context:
    {context}

    User request:
    {query}

    Attached document:
    \"\"\"{attachment}\"\"\"
    """

    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are an expert assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=8192,
        temperature=0.7,
        top_p=0.95,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        model=config["deployment"]
    )

    return response.choices[0].message.content

In [8]:
answer=call_gpt_with_context(
    context="La finance est l'étude de la gestion de l'argent, y compris l'investissement, l'emprunt et l'épargne.",
    query="Explique-moi les bases de la finance.",
    attachment="La finance comprend des concepts tels que les marchés financiers, les instruments financiers, et la gestion des risques.",
    model_name="model-router")
print(answer)

La finance est la discipline qui concerne la gestion de l'argent, incluant l'investissement, l'emprunt et l'épargne. Elle repose sur plusieurs concepts fondamentaux :

1. **Les marchés financiers** : ce sont des lieux ou des plateformes où se réalisent l'achat et la vente d'instruments financiers. Ils permettent aux investisseurs de financer des entreprises ou de se constituer une épargne.

2. **Les instruments financiers** : ce sont les produits échangés sur les marchés, comme les actions, les obligations ou les dérivés. Ils servent à investir, à se couvrir contre les risques ou à financer des projets.

3. **La gestion des risques** : cela consiste à identifier, évaluer et maîtriser les risques liés aux investissements ou aux emprunts, afin de protéger les actifs et optimiser les rendements.

En résumé, la finance inclut l’étude des marchés, des produits financiers et des stratégies pour gérer efficacement l’argent tout en minimisant les risques.


In [7]:
class RAGAppAzure:
    def __init__(self):
        # Clients Azure
        self.search_client = SearchClient(
            endpoint=AZURE_SEARCH_ENDPOINT,
            index_name=INDEX_NAME,
            credential=AzureKeyCredential(AZURE_SEARCH_KEY)
        )
        self.openai_client = AzureOpenAI(
            api_key=AZURE_OPENAI_KEY,
            api_version="2024-12-01-preview",
            azure_endpoint=AZURE_OPENAI_ENDPOINT
        )
        
        # Hack pour l'UI: on simule un buffer vide
        self.buffer = DummyBuffer() 
        print("☁️ RAG App connectée à Azure AI Search")

    def _generate_embedding(self, text):
        response = self.openai_client.embeddings.create(input=[text], model=EMBEDDING_MODEL)
        return response.data[0].embedding

    def query(self, question: str) -> str:
        print(f"🔍 Question: {question}")
        
        # 1. Vectorisation de la question
        vector = self._generate_embedding(question)
        
        # 2. Recherche Vectorielle (Notebook logic)
        vector_query = VectorizedQuery(vector=vector, k_nearest_neighbors=3, fields="contentVector")
        
        results = self.search_client.search(
            search_text=question, # Hybrid search (Text + Vector)
            vector_queries=[vector_query],
            select=["title", "content", "source"],
            top=3
        )
        
        # 3. Construction du contexte
        articles = list(results)
        print(f"📄 Documents trouvés: {len(articles)}")
        
        if not articles:
            return "Désolé, aucune information pertinente trouvée dans la base de connaissances."

        context_str = "\n\n".join([
            f"- Titre: {doc['title']}\n  Contenu: {doc['content']}" 
            for doc in articles
        ])
        
        # 4. Génération (Via Ollama Local - comme avant)
        prompt = f"""Tu es un analyste financier expert. Utilise UNIQUEMENT le contexte ci-dessous pour répondre.
        
        Contexte:
        {context_str}
        
        Question: {question}
        
        Réponse concise:"""
        
        return self._call_ollama(prompt)

    def _call_ollama(self, prompt):
        # (Même code que votre ancien RAGAppKafka)
        try:
            resp = requests.post(
                f"{OLLAMA_BASE_URL}/api/generate",
                json={"model": MODEL_NAME, "prompt": prompt, "stream": False}
            )
            return resp.json().get("response", "Erreur modèle")
        except Exception as e:
            return f"Erreur Ollama: {e}"

In [3]:
from rag_app_azure import RAGAppAzure

def test_call_ollama():
    app = RAGAppAzure()
    prompt = "Explique-moi les bases de la finance."
    response = app._call_ollama(prompt)
    print("Test _call_ollama:")
    print(f"Prompt: {prompt}")
    print(f"Réponse: {response}")

def test_query():
    app = RAGAppAzure()
    question = "Pourquoi Tesla a-t-elle connu une telle volatilité récemment ?"
    response = app.query(question)
    print("Test query:")
    print(f"Question: {question}")
    print(f"Réponse: {response}")

if __name__ == "__main__":
    print("=== Test _call_ollama ===")
    test_call_ollama()
    print("\n=== Test query ===")
    test_query()

=== Test _call_ollama ===
☁️ RAG App connectée à Azure AI Search


KeyboardInterrupt: 

In [14]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
search_client = SearchClient(
            endpoint=AZURE_SEARCH_ENDPOINT,
            index_name=INDEX_NAME,
            credential=AzureKeyCredential(AZURE_SEARCH_KEY))

doc_count = search_client.get_document_count()
print(f"📊 Azure contient {doc_count} documents.")

📊 Azure contient 13 documents.
